<a href="https://colab.research.google.com/github/eternaldemon/Spam-Email-Classifier/blob/master/Spam_Classifier_LSTM%26Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @author:eternal_demon/Bhanu Aggarwal


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,Input,GlobalMaxPooling1D, LSTM,Embedding
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
url = 'https://drive.google.com/file/d/16BBhy4u0QNd8ArilduIVUn3oCraPjivs/view?usp=sharing'
from google.colab import files
files.upload()


In [4]:
data = pd.read_csv('emails.csv')

In [5]:
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [6]:
x = data.iloc[:,:-1]
y = data.iloc[:,1]

In [7]:
print(x.shape)
print(y.shape)

(5728, 1)
(5728,)


In [8]:
y = np.array(y)

In [9]:
y = np.reshape(y,(-1,1))

In [10]:
y = pd.DataFrame(y)

In [11]:
temp =[]
notrequired = [',' , '.' , "Subject" , ':' , '?' , '*' , '&' , '%' , '$' , '#' , '@' , '!' , '~' , '/' ,'-','_','\,'] 

In [ ]:
'''
for i in range(len(x)):
  #print(x.iloc[i,0])
  abc =""
  for word in x.iloc[i,0]:
    if(word not in notrequired):
      abc = abc + word +" "
  temp.append(abc)
  #print(temp[i])
  #print("----------------------------------------------------------------------------------------------------------------")
  '''

In [ ]:
'''
for i in range(len(x)):
  abc=x.iloc[i,0]
  temp.append(abc)
  #print(abc)

'''

In [86]:
'''
for entry in range(len(temp)):
  temp[entry] = temp[entry][13:]
  #print(temp[entry])
  #print("-------------------")

'''


In [12]:
x_train,x_test,y_train,y_test = train_test_split(x['text'],y,test_size=0.2)

In [13]:
# Convert sentences to sequences
MAX_VOCAB_SIZE = 30000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(x_train)
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

In [14]:
#Get word to integer mapping

word2idx = tokenizer.word_index
v = len(word2idx)
print('No of tokens:', v)

No of tokens: 33916


In [15]:
# padding sequences so that we get a N x T matrix

data_train = pad_sequences(sequences_train)
print('Shape of data train tensor:', data_train.shape)

# get sequence length
T = data_train.shape[1]
print(T)

Shape of data train tensor: (4582, 7264)
7264


In [16]:
data_test = pad_sequences(sequences_test, maxlen=T)
print('Shape of data test tensor:', data_test.shape)

Shape of data test tensor: (1146, 7264)


In [41]:
# Create the model

#Choosing embedding dimensionality
D = 20

# Choosing Hidden state dimensionality
M = 15

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

i = Input(shape=(T,))
x = Embedding(v + 1, D)(i)
x = LSTM(M, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)


In [42]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 7264)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 7264, 20)          678340    
_________________________________________________________________
lstm_3 (LSTM)                (None, 7264, 15)          2160      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 15)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 16        
Total params: 680,516
Trainable params: 680,516
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.output_shape

(None, 1)

In [43]:
#model.load_weights('spamfilterweights.h5')

In [44]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [45]:
print('Starting Model Training')
#Remember that here training data name is data_train after preprocessing not x_train
his = model.fit(data_train,y_train,epochs=2,verbose=1,shuffle=False,validation_data=(data_test,y_test))

Starting Model Training
Epoch 1/2
144/144 [==============================] - 780s 5s/step - loss: 0.0175 - accuracy: 0.9998 - val_loss: 0.0338 - val_accuracy: 0.9895
Epoch 2/2
144/144 [==============================] - 766s 5s/step - loss: 0.0072 - accuracy: 0.9998 - val_loss: 0.0334 - val_accuracy: 0.9921


In [49]:
#model.save_weights('spamfilterweights2.h5')

In [58]:
# Validation Loss and Validation Accuracy
model.evaluate(data_test,y_test)

36/36 [==============================] - 13s 354ms/step - loss: 0.0334 - accuracy: 0.9921


[0.03341212123632431, 0.9921466112136841]

In [51]:
val_loss=his.history['val_loss']
val_accuracy=his.history['val_accuracy']
training_loss=his.history['loss']
training_accuracy=his.history['accuracy']
epochs = range(2)

In [ ]:
# Validation Loss per epoch vs Training Loss per epoch

plt.plot(epochs,val_loss,label='Validation Loss')
plt.plot(epochs,training_loss,label='Training Loss')
plt.legend()
plt.show()

In [ ]:
# Validation Accuracy per epoch vs Training accuracy per epoch

plt.plot(epochs,val_accuracy,label='Validation Accuracy')
plt.plot(epochs,training_accuracy,label='Training Accuracy')
plt.legend()
plt.show()